#Feed Forward Neural Network (32x32 Neurons)
To run this notebook, install the [hyperas](https://github.com/maxpumperla/hyperas) dependency and run each cell. This notebook trains the feed forward neural network on the training data, and evaluates it on the test dataset. It saves the model into the `\weights` directory.

In [ ]:
# Install dependencies
!pip install hyperas

In [ ]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from tensorflow.keras.constraints import max_norm
from random import randint
import keras
import numpy as np
from sklearn.utils import shuffle
from hyperopt import Trials, STATUS_OK, tpe
import os
import matplotlib.pyplot as plt

In [ ]:
# Changes to working directory
os.chdir('/content/drive/My Drive/github/NNanobody/regression')

In [ ]:
def create_model(X_train, Y_train, X_test, Y_test):
  """
  create_model compiles and fits the model for five epochs in order for tuning 
  of hyperparameters.

  :param X_train: Independent variable of training data (one hot encoded CDR3 sequences)
  :param Y_train: Dependent variable of training data (enrichment of CDR3 sequences)
  :param X_test: Independent variable of test data (one hot encoded CDR3 sequences)
  :param Y_test: Dependent variable of test data (enrichment of CDR3 sequences)
  :return: Returns the validation loss, hyperopt status, and the model itself after training
  """ 
  
  W_maxnorm = 3 # Normalization of weights to prevent overfit
  DROPOUT = {{choice([0.3,0.5,0.7])}} # Dropout choices for hyperparameters optimization

  # Construct feed-forward neural network (32x32 neurons). Uses leaky ReLU activation function to fix "dying ReLU" problem and speed up training.
  model = Sequential()
  model.add(Flatten(input_shape=(20, 1, 20)))
  model.add(Dense(32,activation='leaky_relu',kernel_constraint=max_norm(W_maxnorm)))
  model.add(Dropout(DROPOUT))
  model.add(Dense(32,activation='leaky_relu',kernel_constraint=max_norm(W_maxnorm)))
  model.add(Dropout(DROPOUT))
  model.add(Dense(1))

  myoptimizer = RMSprop(lr={{choice([1e-1,0.01,0.001,0.0001,1e-5])}}, rho=0.9, epsilon=1e-6) # RMSProp optimizer for the model. Learning rate choices for hyperparameter optimizaiton.
  mylossfunc = 'mean_squared_error'
  model.compile(loss=mylossfunc, optimizer=myoptimizer) # Compile model
  result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1, verbose=False) # Train model for 5 epochs (hyperparameter optimization)

  val_loss = np.amax(result.history['val_loss']) # Save validation loss to return

  return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [ ]:
def data():
  """
  data is a helper function that returns the data necessary for training and validation.

  :return: Returns the shuffled training data and testing data
  """
  
  data_train = h5py.File('./data/Hold out Top 4%/train.h5.batch1', 'r') # Load embedded data from .h5 file
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train) # Shuffle training data

  data_test = h5py.File('./data/Test set Regression/test.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  return X_train_shuffled, Y_train_shuffled, X_test, Y_test

In [ ]:
# Hyperparameter optimization according to the hyperas documentation. Evaluates
# 10 times with the tpe suggested serach method.
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_32_32',
                                          verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



100%|██████████| 10/10 [04:44<00:00, 28.44s/it, best loss: 0.2807278633117676]


In [ ]:
X_train, Y_train, X_test, Y_test = data() # Load all data
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1,verbose=True) # Train the hyperparameter optimized model

Epoch 1/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2618 - val_loss: 0.2396
Epoch 2/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2618 - val_loss: 0.2410
Epoch 3/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2596 - val_loss: 0.2395
Epoch 4/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2599 - val_loss: 0.2406
Epoch 5/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2604 - val_loss: 0.2403
Epoch 6/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2604 - val_loss: 0.2414
Epoch 7/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2594 - val_loss: 0.2395
Epoch 8/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2597 - val_loss: 0.2391
Epoch 9/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2595 - val_loss: 0.2397
Epoch 10/20
586/586 [==============================] - 2s 3ms/step - loss: 0.2590 - val_loss: 0.2390

In [ ]:
best_model.evaluate(X_test, Y_test) # Evaluate model on the test set

710/710 [==============================] - 1s 2ms/step - loss: 0.5811


0.5811499357223511

In [ ]:
best_model.save('../weights/regression/Hold out Top 4%/seq_32_32') # Save the model

INFO:tensorflow:Assets written to: ../weights/regression/Hold out Top 4%/seq_32_32/assets
